In [ ]:
# Importando bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
dados = pd.read_csv('student-mat.csv', sep= ';')
dados.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [ ]:
# remover as instâncias com valores faltantes do DataFrame
dados = dados.dropna()

In [ ]:
#remover instâncias duplicadas
dados = dados.drop_duplicates()

In [ ]:
# Criando o dataframe Y com a coluna G3
Y = dados[['G3']]

In [ ]:
# Criando o dataframe X_possiveis com todas as colunas do dataframe original, exceto 'G2' e 'G3'
X_possiveis = dados.drop(['G2', 'G3'], axis=1)

In [ ]:
# Selecionando variáveis para X (exemplo)
X = X_possiveis[['sex', 'age', 'Medu', 'Fedu', 'studytime', 'failures', 'absences', 'G1']]

In [ ]:
# Aplicando One-Hot Encoding para as variáveis categóricas, se houver
X = pd.get_dummies(X, columns=['sex'])

In [ ]:
# Define a precisão de exibição para 2 casas decimais e descreve a tabela
pd.set_option('display.precision', 2)
X.describe()

,age,Medu,Fedu,studytime,failures,absences,G1,sex_F,sex_M
count,395.00,395.00,395.00,395.00,395.00,395.00,395.00,395.00,395.00
mean,16.70,2.75,2.52,2.04,0.33,5.71,10.91,0.53,0.47
std,1.28,1.09,1.09,0.84,0.74,8.00,3.32,0.50,0.50
min,15.00,0.00,0.00,1.00,0.00,0.00,3.00,0.00,0.00
25%,16.00,2.00,2.00,1.00,0.00,0.00,8.00,0.00,0.00
50%,17.00,3.00,2.00,2.00,0.00,4.00,11.00,1.00,0.00
75%,18.00,4.00,3.00,2.00,0.00,8.00,13.00,1.00,1.00
max,22.00,4.00,4.00,4.00,3.00,75.00,19.00,1.00,1.00


In [ ]:
X.dtypes

age          int64
Medu         int64
Fedu         int64
studytime    int64
failures     int64
absences     int64
G1           int64
sex_F        uint8
sex_M        uint8
dtype: object

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 0 to 394
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        395 non-null    int64
 1   Medu       395 non-null    int64
 2   Fedu       395 non-null    int64
 3   studytime  395 non-null    int64
 4   failures   395 non-null    int64
 5   absences   395 non-null    int64
 6   G1         395 non-null    int64
 7   sex_F      395 non-null    uint8
 8   sex_M      395 non-null    uint8
dtypes: int64(7), uint8(2)
memory usage: 25.5 KB


Separando o conjunto de treinamento do conjunto de teste

In [ ]:
# Separando o conjunto de treinamento do conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

Aplicando One_Hot Encoding para as variaveis categoricas

In [ ]:
#Aplicando One_Hot Encoding para as variaveis categoricas
X_possiveis = pd.get_dummies(X_possiveis, columns=['sex','Pstatus','schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic'])
X_possiveis.head()

,school,age,address,famsize,Medu,Fedu,Mjob,Fjob,reason,guardian,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,GP,18,U,GT3,4,4,at_home,teacher,course,mother,...,1,0,0,1,0,1,1,0,1,0
1,GP,17,U,GT3,1,1,at_home,other,course,father,...,1,0,1,0,0,1,0,1,1,0
2,GP,15,U,LE3,1,1,at_home,other,other,mother,...,1,0,0,1,0,1,0,1,1,0
3,GP,15,U,GT3,4,2,health,services,home,mother,...,0,1,0,1,0,1,0,1,0,1
4,GP,16,U,GT3,3,3,other,other,home,father,...,1,0,0,1,0,1,1,0,1,0


Escalando Dados, criando os modelos sem escala.

In [ ]:
# Escalando os dados
scaler = MinMaxScaler()
X_train_sem_escala = X_train.copy()
X_test_sem_escala = X_test.copy()

# Normalizando os dados de treinamento e teste
scaler.fit(X_train_sem_escala)
X_train_sem_escala = scaler.transform(X_train_sem_escala)
X_test_sem_escala = scaler.transform(X_test_sem_escala)

# Criando o modelo de Regressão Linear sem escalar os dados
modelo_regressao_linear_sem_escala = LinearRegression()
modelo_regressao_linear_sem_escala.fit(X_train_sem_escala, y_train)
y_pred_regressao_linear_sem_escala = modelo_regressao_linear_sem_escala.predict(X_test_sem_escala)
rmse_regressao_linear_sem_escala = mean_squared_error(y_test, y_pred_regressao_linear_sem_escala, squared=False)
r2_regressao_linear_sem_escala = r2_score(y_test, y_pred_regressao_linear_sem_escala)
mae_regressao_linear_sem_escala = mean_absolute_error(y_test, y_pred_regressao_linear_sem_escala)
mse_regressao_linear_sem_escala = mean_squared_error(y_test, y_pred_regressao_linear_sem_escala)

# Criando o modelo de Árvore de Decisão para Regressão sem escalar os dados
modelo_arvore_decisao_sem_escala = DecisionTreeRegressor()
modelo_arvore_decisao_sem_escala.fit(X_train_sem_escala, y_train)
y_pred_arvore_decisao_sem_escala = modelo_arvore_decisao_sem_escala.predict(X_test_sem_escala)
rmse_arvore_decisao_sem_escala = mean_squared_error(y_test, y_pred_arvore_decisao_sem_escala, squared=False)
r2_arvore_decisao_sem_escala = r2_score(y_test, y_pred_arvore_decisao_sem_escala)
mae_arvore_decisao_sem_escala = mean_absolute_error(y_test, y_pred_arvore_decisao_sem_escala)
mse_arvore_decisao_sem_escala = mean_squared_error(y_test, y_pred_arvore_decisao_sem_escala)

Escalando Dados, criando os modelos sem escala.

In [ ]:
# Escalando os dados
scaler = MinMaxScaler()
X_train_sem_escala = X_train.copy()
X_test_sem_escala = X_test.copy()

# Normalizando os dados de treinamento e teste
scaler.fit(X_train_sem_escala)
X_train_sem_escala = scaler.transform(X_train_sem_escala)
X_test_sem_escala = scaler.transform(X_test_sem_escala)

# Criando o modelo de Regressão Linear escalando os dados
modelo_regressao_linear_com_escala = LinearRegression()
modelo_regressao_linear_com_escala.fit(X_train, y_train)
y_pred_regressao_linear_com_escala = modelo_regressao_linear_com_escala.predict(X_test)
rmse_regressao_linear_com_escala = mean_squared_error(y_test, y_pred_regressao_linear_com_escala, squared=False)
r2_regressao_linear_com_escala = r2_score(y_test, y_pred_regressao_linear_com_escala)
mae_regressao_linear_com_escala = mean_absolute_error(y_test, y_pred_regressao_linear_com_escala)
mse_regressao_linear_com_escala = mean_squared_error(y_test, y_pred_regressao_linear_com_escala)

# Criando o modelo de Árvore de Decisão para Regressão escalando os dados
modelo_arvore_decisao_com_escala = DecisionTreeRegressor()
modelo_arvore_decisao_com_escala.fit(X_train, y_train)
y_pred_arvore_decisao_com_escala = modelo_arvore_decisao_com_escala.predict(X_test)
rmse_arvore_decisao_com_escala = mean_squared_error(y_test, y_pred_arvore_decisao_com_escala, squared=False)
r2_arvore_decisao_com_escala = r2_score(y_test, y_pred_arvore_decisao_com_escala)
mae_arvore_decisao_com_escala = mean_absolute_error(y_test, y_pred_arvore_decisao_com_escala)
mse_arvore_decisao_com_escala = mean_squared_error(y_test, y_pred_arvore_decisao_com_escala)


Apresentação dos Dados:

In [ ]:
print("Regressão Linear sem escala:")
print("MAE: {:.3f}".format(mae_regressao_linear_sem_escala))
print("MSE: {:.3f}".format(mse_regressao_linear_sem_escala))
print("RMSE: {:.3f}".format(rmse_regressao_linear_sem_escala))
print("R2: {:.3f}".format(r2_regressao_linear_sem_escala))
print()

print("Regressão Linear com escala:")
print("MAE: {:.3f}".format(mae_regressao_linear_com_escala))
print("MSE: {:.3f}".format(mse_regressao_linear_com_escala))
print("RMSE: {:.3f}".format(rmse_regressao_linear_com_escala))
print("R2: {:.3f}".format(r2_regressao_linear_com_escala))
print()

print("Árvore de Decisão para Regressão com escala:")
print("MAE: {:.3f}".format(mae_arvore_decisao_com_escala))
print("MSE: {:.3f}".format(mse_arvore_decisao_com_escala))
print("RMSE: {:.3f}".format(rmse_arvore_decisao_com_escala))
print("R2: {:.3f}".format(r2_arvore_decisao_com_escala))
print()

print("Árvore de Decisão para Regressão sem escala:")
print("MAE: {:.3f}".format(mae_arvore_decisao_sem_escala))
print("MSE: {:.3f}".format(mse_arvore_decisao_sem_escala))
print("RMSE: {:.3f}".format(rmse_arvore_decisao_sem_escala))
print("R2: {:.3f}".format(r2_arvore_decisao_sem_escala))


Regressão Linear sem escala:
MAE: 1.951
MSE: 6.807
RMSE: 2.609
R2: 0.690

Regressão Linear com escala:
MAE: 1.951
MSE: 6.807
RMSE: 2.609
R2: 0.690

Árvore de Decisão para Regressão com escala:
MAE: 2.244
MSE: 11.303
RMSE: 3.362
R2: 0.486

Árvore de Decisão para Regressão sem escala:
MAE: 2.370
MSE: 13.328
RMSE: 3.651
R2: 0.394


A Regressão Linear apresenta resultados semelhantes com e sem escala, indicando que a escala dos dados não teve um impacto significativo na performance do modelo.
A Árvore de Decisão para Regressão, por outro lado, mostra uma diferença no desempenho com e sem escala, com resultados ligeiramente melhores quando os dados são escalados. Isso sugere que a escala dos dados pode ter um impacto mais notável no desempenho de modelos baseados em árvores de decisão.
Em termos de métricas individuais, a Regressão Linear apresenta valores mais baixos de RMSE, MAE e MSE, indicando um melhor ajuste aos dados em comparação com a Árvore de Decisão para Regressão. No entanto, a Árvore de Decisão para Regressão tem um R2 ligeiramente maior do que a Regressão Linear com escala, o que sugere uma melhor capacidade de explicar a variação nos dados.
A Regressão Linear parece ser a escolha mais favorável no geral para este conjunto de dados específico.
